In [ ]:
# 런타임 연결 해제 및 삭제

In [ ]:
# 이 부분은 처음 한번만 실행하면 됌.
# 코드 수정 - "The reason is that the last Ubuntu update update supports chromium driver just via snap."
# 최근 우분투 업데이트에서 크롬 드라이버 설치를 snap을 이용해서만 하도록 바뀜
# 고로 snap 없이 설치하는 아래 우회 코드로 변경
# 출처 : https://colab.research.google.com/drive/1cbEvuZOhkouYLda3RqiwtbM-o9hxGLyC
# 출처2 : https://stackoverflow.com/questions/75155063/selenium-use-chrome-on-colab-got-unexpectedly-exited

%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument('--disable-dev-shm-usage')
options.add_argument('lang=ko_KR')
options.headless = True

<ipython-input-3-8e2c29e82798>:10: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = True


In [4]:
with webdriver.Chrome("/usr/bin/chromedriver", options=options) as driver:

  driver.get('https://www.coffeebeankorea.com/store/store.asp')
  html = driver.page_source
  soup = BeautifulSoup(html, 'html.parser')
  temp = soup.select('#storeListUL > li')
  for i in temp:
    print(i.select('div.store_txt > p.name > span')[0].text)


# # 사용후 종료
# driver.quit()

<ipython-input-4-a3de951bfe0c>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  with webdriver.Chrome("/usr/bin/chromedriver", options=options) as driver:


삼성봉은사거리점33m
코엑스인터점262m
삼성오크우드점316m
삼성봉은사로점442m
도심공항타워점495m
트레이드타워점507m
도심공항점560m
파르나스몰점626m
삼성로점632m
삼성루첸타워점845m


In [5]:
# 코드 2
import time
with webdriver.Chrome("/usr/bin/chromedriver", options=options) as driver:  
  
  driver.get('https://www.coffeebeankorea.com/store/store.asp')
  time.sleep(1)

  driver.execute_script("storePop2(31)")  
  time.sleep(1)


  html = driver.page_source
  soup = BeautifulSoup(html, 'html.parser')  
  
  tbody_list = soup.select('#matizCoverLayer0Content > div > div > div.store_txt > table > tbody > tr')
  print(tbody_list)
  for row in tbody_list:    
    print(row.find('td').text)  

# 다른 매장 검색
  driver.get('https://www.coffeebeankorea.com/store/store.asp')
  time.sleep(1)
  driver.execute_script("storePop2(142)")
  time.sleep(1)

  html = driver.page_source
  soup = BeautifulSoup(html, 'html.parser')  

  #matizCoverLayer0Content > div > div > div.store_txt > table > tbody
  tbody_list = soup.select('#matizCoverLayer0Content > div > div > div.store_txt > table > tbody > tr')
  print(tbody_list)
  for row in tbody_list:    
    print(row.find('td').text)  

<ipython-input-5-26a15e0cc261>:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  with webdriver.Chrome("/usr/bin/chromedriver", options=options) as driver:


[<tr> <th>영업시간</th> <td>평일 07:00-22:00 | 주말/공휴일 08:00-22:00</td> </tr>, <tr> <th>주차</th> <td>건물뒷편주차장(기계주차식) 평일 최초 30분 2,000원 / 1시간 5,000원 (구매영수증 지참시 30분 2,000원 / 1시간 3,500원 1시간 초과시 10분당 1,000원 ( 평일,주말 동일 )</td> </tr>, <tr> <th>주소</th> <td>서울시 강남구 영동대로 607 1,2층  <!--span class="lot">(서울시 강남구 영동대로 607 1,2층)</span--></td> </tr>, <tr> <th>전화번호</th> <td>02-3443-5618</td> </tr>, <tr> <th rowspan="2">기타 정보</th> <td class="best">식약처 인증 위생등급 매우 우수매장</td> </tr>, <tr> <td class="hallcake">홀케익 당일 수령가능</td> </tr>]
평일 07:00-22:00 | 주말/공휴일 08:00-22:00
건물뒷편주차장(기계주차식) 평일 최초 30분 2,000원 / 1시간 5,000원 (구매영수증 지참시 30분 2,000원 / 1시간 3,500원 1시간 초과시 10분당 1,000원 ( 평일,주말 동일 )
서울시 강남구 영동대로 607 1,2층  
02-3443-5618
식약처 인증 위생등급 매우 우수매장
홀케익 당일 수령가능
[<tr> <th>영업시간</th> <td>평일 07:00~21:00ㅣ주말/공휴일 07:00~21:00</td> </tr>, <tr> <th>주차</th> <td>주차불가</td> </tr>, <tr> <th>주소</th> <td>서울시 강남구 봉은사로 524 코엑스 인터컨티넨탈아케이드  <!--span class="lot">(서울시 강남구 봉은사로 524 코엑스 인터컨티넨탈아케이드)</span--></td> </tr>, <tr> <th>전화번호</th> <td>02-569-3103</t

In [14]:
# 코드3  https://www.koreabaseball.com/TeamRank/TeamRank.aspx
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
import time
with webdriver.Chrome("/usr/bin/chromedriver", options=options) as driver:  
  driver.get('https://www.koreabaseball.com/TeamRank/TeamRank.aspx')  
  
  year_dropdown = Select(driver.find_element(By.ID,'cphContents_cphContents_cphContents_ddlYear') )
  year_dropdown.select_by_visible_text('2020')
  time.sleep(1) 
  html = driver.page_source
  soup = BeautifulSoup(html, 'html.parser')    
  tr_lists = soup.select('#cphContents_cphContents_cphContents_udpRecord > table > tbody > tr')
  for row in tr_lists:
    print( [td.text for td in row.find_all('td')] )
  
  print('*'*50,'2022','*'*50)
  year_dropdown = Select(driver.find_element(By.ID,'cphContents_cphContents_cphContents_ddlYear') )
  year_dropdown.select_by_visible_text('2022')
  time.sleep(1) 
  html = driver.page_source
  soup = BeautifulSoup(html, 'html.parser')    
  tr_lists = soup.select('#cphContents_cphContents_cphContents_udpRecord > table > tbody > tr')
  for row in tr_lists:
    print( [td.text for td in row.find_all('td')] )


<ipython-input-14-2f879657ab23>:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  with webdriver.Chrome("/usr/bin/chromedriver", options=options) as driver:


['1', 'NC', '144', '83', '55', '6', '0.601', '0', '4승2무4패', '1패', '45-3-24', '38-3-31']
['2', '두산', '144', '79', '61', '4', '0.564', '5', '6승0무4패', '4승', '41-2-29', '38-2-32']
['3', 'KT', '144', '81', '62', '1', '0.566', '4.5', '7승0무3패', '1패', '44-0-28', '37-1-34']
['4', 'LG', '144', '79', '61', '4', '0.564', '5', '5승1무4패', '2패', '39-3-30', '40-1-31']
['5', '키움', '144', '80', '63', '1', '0.559', '5.5', '6승0무4패', '1패', '41-1-30', '39-0-33']
['6', 'KIA', '144', '73', '71', '0', '0.507', '13', '4승0무6패', '1승', '37-0-35', '36-0-36']
['7', '롯데', '144', '71', '72', '1', '0.497', '14.5', '3승0무7패', '1승', '41-0-31', '30-1-41']
['8', '삼성', '144', '64', '75', '5', '0.460', '19.5', '6승2무2패', '1승', '33-2-37', '31-3-38']
['9', 'SK', '144', '51', '92', '1', '0.357', '34.5', '5승0무5패', '1승', '28-0-44', '23-1-48']
['10', '한화', '144', '46', '95', '3', '0.326', '38.5', '3승1무6패', '1승', '25-2-45', '21-1-50']
****************************************************************************************************


In [8]:
# # 사용후 종료
driver.quit()